In [2]:
import sys
import os
import pandas as pd

# Adiciona o caminho raiz do projeto ao sys.path
caminho_raiz = os.path.abspath("Z:\\Chila\\projetos\\calculadora_titulos_publicos")
if caminho_raiz not in sys.path:
    sys.path.append(caminho_raiz)

from titulospub.dados.ipca import inicio_fim_mes_ipca
from titulospub.utils.datas import dias_trabalho_total, data_vencimento_ajustada
from titulospub.utils.carregamento_var_globais import _carrecar_ipca_dict_se_necessario, _carregar_feriados_se_necessario
from titulospub.dados.orquestrador import VariaveisMercado
from titulospub.core.auxilio import codigo_vencimento_bmf

In [ ]:
b32 = NTNB("2032-08-15", taxa=7.651)



✅ Usando cache existente de ANBIMAS completo.


In [3]:
vm = VariaveisMercado()
vm.atualizar_tudo()

🔄 Atualizando variáveis de mercado...
📡 Buscando feriados via scraping...
📡 Calculando IPCA dict...
📡 Buscando CDI...
📡 Realizando scraping ANBIMA...
♻️ Cache salvo para todos os títulos ANBIMA.
📡 Realizando scraping BMF...
♻️ Cache salvo para todos os contrados de DI e DAP.
📡 Realizando scraping VNA_LFT...
♻️ Cache salvo para VNA_LFT.
✅ Atualização concluída.


In [11]:
ipca_dict = vm.get_ipca_dict()
ipca_dict

{'ULTIMO_MES_IPCA': 9,
 'INDICE_IPCA_DATA_BASE': 1614.62,
 'INDICE_IPCA_FECHADO_ATUAL': 7359.06,
 'INDICE_IPCA_FECHADO_ANTERIOR': 7323.91,
 'VAR_IPCA_ATUAL': 0.48,
 'VAR_IPCA_ANTERIOR': -0.11,
 'IPCA_PROJ': 0.21,
 'IPCA_USADO': 0.4799348981623375}

In [3]:
i, f = inicio_fim_mes_ipca(pd.Timestamp.today().normalize())

In [12]:
def dia_15_do_mes(data: pd.Timestamp) -> pd.Timestamp:
    """Retorna o dia 15 do mês e ano da data fornecida."""
    return pd.Timestamp(year=data.year, month=data.month, day=15)

def calculo_prt(data=None, ipca_dict=None):
    if data == None:
        data = pd.Timestamp.today().normalize()
    
    ipca_dict = _carrecar_ipca_dict_se_necessario(ipca_dict)

    i, f = inicio_fim_mes_ipca(pd.Timestamp.today().normalize())
    
    pro_rata = ipca_dict["INDICE_IPCA_FECHADO_ATUAL"]
    ipca_usado = ipca_dict["IPCA_USADO"]
    dias_totais = dias_trabalho_total(i,f)
    dias_passados = dias_trabalho_total(i, data)

    return round(pro_rata * ((1 + ipca_usado / 100) ** (dias_passados / dias_totais)), 2)

def calculo_pu_dap(taxa: float, codigo: str=None, data_liquidacao=None, data_vencimento:pd.Timestamp=None, feriados: list=None):
        feriados = _carregar_feriados_se_necessario(feriados)  
        if data_liquidacao == None: data_liquidacao=pd.Timestamp.today().normalize()
        
        if codigo == None and data_vencimento == None:
            raise ValueError("Fornece o codigo ou vencimento")
        elif codigo == None:
            # Já tem data_vencimento fornecida
            pass
        elif data_vencimento == None:
            # Obtém data do código
            data_vencimento = codigo_vencimento_bmf(codigo)
        else:
            # Ambos fornecidos, usa data_vencimento
            pass
        
        # Converte para o dia 15 do mês ANTES de ajustar
        data_vencimento = dia_15_do_mes(data_vencimento)
        
        # Ajusta para dia útil
        data_vencimento = data_vencimento_ajustada(data=data_vencimento, feriados=feriados)
        
        dias_uteis = dias_trabalho_total(data_liquidacao, data_vencimento)
        return 100000 / ((taxa / 100 +1) ** (dias_uteis / 252))
    
def calculo_financeiro_dap(taxa: float, codigo: str=None, data_liquidacao=None, data_vencimento:pd.Timestamp=None, feriados: list=None):
    pu = calculo_pu_dap(taxa=taxa, 
                        codigo=codigo, 
                        data_liquidacao=data_liquidacao,
                        data_vencimento=data_vencimento,
                        feriados=feriados)
    
    prt = calculo_prt(data=data_liquidacao)
    return pu * 0.00025 * prt

In [15]:
calculo_financeiro_dap(taxa=7.85, codigo="DAPQ32")

np.float64(110491.5049863021)

In [18]:
calculo_prt()

np.float64(7389.55)